# Day 10 Part 1 Streaming+basics

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType , IntegerType, FloatType

schema = StructType([
                     StructField('Country',StringType()),
                     StructField('Citizens',IntegerType())
])

In [ ]:
source_dir = 'dbfs:/FileStore/streaming/'

In [ ]:
%sql

CREATE SCHEMA IF NOT EXISTS  stream;
use stream

In [ ]:
# spark.readStream Returns a DataStreamReader that can be used to read data streams as a streaming DataFrame.
df = spark.readStream.format("csv")\
        .option('header','true')\
        .schema(schema)\
        .load(source_dir)

In [ ]:
display(df)

Country,Citizens
India,10
USA,5
China,10
India,10
Canada,40
Brazil,10
India,5
USA,10
China,5
India,5


In [ ]:
# Complete code Resource: https://spark.apache.org/docs/3.5.3/structured-streaming-programming-guide.html
 WriteStream = ( df.writeStream
        .option('checkpointLocation',f'{source_dir}/AppendCheckpoint')
        .outputMode("append")
        .queryName('AppendQuery')
        .toTable("stream.AppendTable"))

In [ ]:
%sql
SELECT * FROM stream.AppendTable

Country,Citizens
India,10
USA,5
China,10
India,10
Canada,40
Brazil,10
India,5
USA,10
China,5
India,5


In [ ]:
WriteStream.stop()

## Day 10 Part 2 outputModes

In [ ]:
dbutils.fs.rm('dbfs:/user/hive/warehouse/stream.db',True)
dbutils.fs.rm('dbfs:/FileStore/streaming',True)

In [ ]:
dbutils.fs.rm('dbfs:/FileStore/streaming/CompleteCheckpoint',True)

In [ ]:
%sql
DROP DATABASE IF EXISTS stream CASCADE;
CREATE DATABASE IF NOT EXISTS stream

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType , IntegerType, FloatType

schema = StructType([
                     StructField('Country',StringType()),
                     StructField('Citizens',IntegerType())
])

In [ ]:
source_dir = 'dbfs:/FileStore/streaming/'

In [ ]:
df = spark.readStream.format("csv")\
        .option('header','true')\
        .schema(schema)\
        .load(source_dir)

### Append

In [ ]:
 WriteStream = ( df.writeStream
        .option('checkpointLocation',f'{source_dir}/AppendCheckpoint')
        .outputMode("append")
        .queryName('AppendQuery')
        .toTable("stream.AppendTable"))

In [ ]:
%sql
SELECT * FROM stream.AppendTable

## Complete

In [ ]:
from pyspark.sql.functions import sum
df_complete = df.groupBy('Country').agg(sum('Citizens').alias('Total_Population'))

In [ ]:
 WriteCompleteStream = ( df_complete.writeStream
        .option('checkpointLocation',f'{source_dir}/CompleteCheckpoint')
        .outputMode("complete")
        .queryName('CompleteQuery')
        .toTable("stream.CompleteTable"))

In [ ]:
%sql
SELECT * FROM stream.CompleteTable

## Day 10 Part 3 Triggers

#### Reading the streaming dataframe

In [ ]:
df = spark.readStream.format("csv")\
        .option('header','true')\
        .schema(schema)\
        .load(source_dir)


#### 01. Trigger - default or unspecifed Trigger

In [ ]:
 WriteStream = ( df.writeStream
        .option('checkpointLocation',f'{source_dir}/AppendCheckpoint')
        .outputMode("append")
        .queryName('DefaultTrigger')
        .toTable("stream.AppendTable"))


#### 02. Trigger - processingTime

In [ ]:
 WriteStream = ( df.writeStream
        .option('checkpointLocation',f'{source_dir}/AppendCheckpoint')
        .outputMode("append")
        .trigger(processingTime='2 minutes')
        .queryName('ProcessingTime')
        .toTable("stream.AppendTable"))


#### 03. Trigger - availablenow

In [ ]:
 WriteStream = ( df.writeStream
        .option('checkpointLocation',f'{source_dir}/AppendCheckpoint')
        .outputMode("append")
        .trigger(availableNow=True)
        .queryName('AvailableNow')
        .toTable("stream.AppendTable"))

In [ ]:
%sql
SELECT * FROM stream.AppendTable

In [ ]:
%sql
drop table stream.AppendTable